https://ocw.mit.edu/courses/15-988-system-dynamics-self-study-fall-1998-spring-1999/7ac2f07c6b562211becc8afb0102cf88_modeling2.pdf

(equations are listed and documented in the appendix)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import reno

In [ ]:
b = reno.Model("Business sector")
b.structures = reno.Stock(init=1000, doc="Number of business structures in the city.")
b.construction = reno.Flow(doc="Rate of construction of business structures. It is affected by the number of already existing business structures, a normal construction fraction, and the availability of land and labor.")
b.demolition = reno.Flow(doc="Rate of demolition of business structures.")

b.average_structure_lifetime = reno.Variable(50, doc="Average lifetime of a business structure.")
b.construction_fraction = reno.Variable(0.02, doc="Normal rate of construction of business structure per existing business structure.")
b.jobs = reno.Variable(doc="Number of jobs provided by the existing business structures. It is the product of the number of business structures and the average number of jobs per structure.")
b.jobs_per_structure = reno.Variable(20, doc="The number of jobs provided by each structure.")
b.labor_availability = reno.Variable(doc="The ratio between the labor force and the number of available jobs.")
b.land_area = reno.Variable(5000, doc="Total land area available for commercial development.")
b.land_fraction_occupied = reno.Variable(doc="The fraction of commercial land that has already been developed.")
b.land_per_structure = reno.Variable(1, doc="The amount of land required by each business structure.")

b.labor_availability_multiplier = reno.Variable(
    reno.interpolate(
        b.labor_availability, 
        [0.0, 0.2, 0.4, 0.6, 0.8, 1.00, 1.20, 1.4, 1.6, 1.8, 2.0],
        [0.05, .105, .225, .36, .54, .84, 1.24, 2.36, 3.34, 3.86, 4.0]
    )
)

b.land_availability_multiplier = reno.Variable(
    reno.interpolate(
        b.land_fraction_occupied,
        [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        [1.0, 2.3, 2.98, 3.34, 3.48, 3.5, 3.44, 3.12, 2.3, 1.0, 0.0]
    )
)

b.structures += b.construction
b.structures -= b.demolition
b.construction.eq = b.structures * b.construction_fraction * b.labor_availability_multiplier * b.land_availability_multiplier
b.demolition.eq = b.structures / b.average_structure_lifetime

b.jobs.eq = b.structures * b.jobs_per_structure
b.land_fraction_occupied.eq = b.structures * b.land_per_structure / b.land_area


In [ ]:
p = reno.Model("Population sector")
p.population = reno.Stock(init=50_000, doc="The number of people living in the urban area.")
p.in_migration = reno.Flow(doc="The number of people who move into the urban area each year. It is affected by the current population, a normal fraction of in-migration, and the availability of jobs.")
p.births = reno.Flow(doc="The number of people born in the area per year.")
p.out_migration = reno.Flow(doc="The number of people who leave the urban area each year.")
p.deaths = reno.Flow(doc="Number of people who die each year.")

p.average_lifetime = reno.Variable(66.7, doc="The average lifetime of a person living in the urban area is approximately 67 years.")
p.birth_fraction = reno.Variable(0.015, doc="The fraction of the population that reproduces each year.")
p.in_migration_normal = reno.Variable(0.08, doc="The fraction of the population that immigrates each year under normal conditions.")
p.labor_force = reno.Variable(doc="The number of people who are eligible to work. It is a constant fraction of the population.")
p.labor_participation_fraction = reno.Variable(0.35, doc="The fraction of the total population that is willing and able to work.")
p.out_migration_fraction = reno.Variable(0.08, doc="The fraction of the population that emigrates each year.")

p.job_attractiveness_multiplier = reno.Variable(
    reno.interpolate(
        b.labor_availability,
        [0.0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0],
        [4.0, 3.95, 3.82, 3.56, 2.86, 1.24, 0.64, 0.32, 0.18, 0.105, 0.075]
    ),
    doc="The multiplier shows the effect of labor availability on immigration. When there are many available jobs (labor availability is less than 1), people are inclined to move to the city. When there are not enough available jobs (labor availability is greater than 1), people tend not to immigrate to the urban area."
)

p.population += p.in_migration
p.population += p.births
p.population -= p.out_migration
p.population -= p.deaths

p.in_migration.eq = p.population * p.in_migration_normal * p.job_attractiveness_multiplier
p.births.eq = p.population * p.birth_fraction
p.out_migration.eq = p.population * p.out_migration_fraction
p.deaths.eq = p.population / p.average_lifetime

p.labor_force.eq = p.population * p.labor_participation_fraction



In [ ]:
b.labor_availability.eq = p.labor_force / b.jobs

In [ ]:
urban_growth = reno.Model()
urban_growth.business = b
urban_growth.population = p

In [ ]:
urban_growth.graph()

In [ ]:
urban_growth.latex()

In [ ]:
base_run = urban_growth(steps=80)
base_run

In [ ]:
reno.plot_trace_refs(urban_growth, [base_run], [b.structures, p.population, b.labor_availability], figsize=(15, 5))

In [ ]:
reno.plot_refs_single_axis(base_run, [b.structures, p.population, b.labor_availability], figsize=(15, 5))

In [ ]:
base_run.population_population.values[0][-1], base_run.business_structures.values[0][-1]

In [ ]:
t = reno.TimeRef()
urban_growth.business.land_area.eq = reno.Piecewise([5000, 7500], [t < 10, t >= 10])
urban_growth.business.structures.init = 4499
urban_growth.population.population.init = 277533
more_land = urban_growth(steps=80)
more_land

In [ ]:
reno.plot_trace_refs(urban_growth, {"base": base_run, "more_land": more_land}, [b.structures, p.population, b.labor_availability], figsize=(15, 5))

In [ ]:
reno.plot_refs_single_axis(more_land, [b.structures, p.population, b.labor_availability], figsize=(15, 5))